 ## Conversion script

 Starts from a file `input/test-lime-data.csv`, which is the content of the `RAW` tab of the Excel file shared. Any changes to the column _shape_ of this file will break the script: e.g. edits to the survey.
 
 For each row (a response) in the original data (`df` below), it creates one or many new rows in an output table (`ndf` below).

 It currently does not work through any columns in the _Other_ category for _type_ of relationship.

In [18]:

import numpy as np
import pandas as pd
df = pd.read_csv('input/test-lime-data.csv')
df.index.name = 'id'

ndf = pd.DataFrame(columns=['from_org', 'to_org', 'rel_type', 'rel_freq'])

In [19]:
rel_type_column_names = ['agriculture', 'climate_change', 'community_development', 'conservation', 'economic_development', 'education', 'energy',
                         'food_security', 'gender', 'grant_making', 'health', 'human_rights', 'peace_building', 'policy_making_governance', 'water', 'wildlife_biodiversity']
freq_column_names = ['not_in_6_months', 'once_6_monthly',
                     'multiple_6_monthly', 'multiple_monthly', 'weekly', 'multiple_weekly']
# Column indices
max_partner_length = 10
name_start_idx = [43, 44, 45, 46, 47, 48, 49, 50, 51, 52]
name_len = 1
type_start_idx = [53, 75, 97, 119, 141, 163, 185, 207, 229, 251]
type_len = 16  # 22
freq_start_idx = [273, 279, 285, 291, 297, 303, 309, 315, 321, 327]
freq_len = 6



In [20]:
def flatten_type(row: pd.Series, start_col):
    types = row[start_col:start_col+type_len]
    types.index = rel_type_column_names
    types_keep = types[types == 'Yes']
    res = '|'.join(types_keep.index)
    return res


In [21]:
def flatten_freq(row, start_col):
    freqs = row[start_col:start_col+freq_len]
    freqs.index = freq_column_names
    freq = freqs[freqs == 'Yes']
    return freq.index[0]

In [22]:
# For every row...
for index, row in df.iterrows():
    # Go through every potential partner...
    for partner_index in range(0, max_partner_length):
        partner_number = partner_index + 1
        # print(f'Doing partner {partner_number}')
        from_org = row.iloc[0]

        to_org_idx = name_start_idx[partner_index]
        to_org = row.iloc[to_org_idx]

        if to_org == 'No' or pd.isnull(to_org):
            # print(f'drop!: {to_org}')
            continue

        type_start = type_start_idx[partner_index]
        rel_type = flatten_type(row, type_start)

        freq_start = freq_start_idx[partner_index]
        rel_freq = flatten_freq(row, freq_start)

        row_data = {'from_org': from_org, 'to_org': to_org,
                    'rel_type': rel_type, 'rel_freq': rel_freq}
        ndf = ndf.append(row_data, ignore_index=True)
ndf

,from_org,to_org,rel_type,rel_freq
0,Resource Africa,Southern African Trust (SAT),community_development|health|water,weekly
1,Resource Africa,Community Based Natural Resources Management (...,climate_change|community_development|health|water,weekly
2,Southern Africa Trust,Southern Africa People's Solidarity Network,gender|peace_building|wildlife_biodiversity,multiple_6_monthly
3,Southern Africa Trust,Graca Machel Trust,agriculture|climate_change|conservation|gender...,once_6_monthly
4,Southern Africa Trust,Mandela Institute for Development Studies,conservation|energy|food_security|health|peace...,multiple_6_monthly
5,Southern Africa Trust,SADC Secretariat,agriculture|conservation|energy|gender|health|...,multiple_6_monthly
6,Southern Africa Trust,Centre on African Philanthropy and Social Inve...,conservation|energy|peace_building,once_6_monthly
7,Southern Africa Trust,Foreign and Commonwealth Development Office,agriculture|conservation|energy|gender,multiple_6_monthly
8,Southern Africa Trust,Ford Foundation,agriculture|conservation|energy|food_security|...,multiple_6_monthly
9,Southern Africa Trust,Mott Foundation,climate_change|gender|wildlife_biodiversity,once_6_monthly


In [23]:
ndf.to_csv('output/graph1.csv')